# In Class Notebook, Week 14

You can always paste the URL of this notebook (https://github.com/UIUC-iSchool-DataViz/is445_spring2022/blob/master/week14/inClass_week14.ipynb ) into the nbviewer interface for a plain-text rendering:

https://kokes.github.io/nbviewer.js/viewer.html

In [1]:
import numpy as np
import pandas as pd
import ipyvolume

In [2]:
x,y,z = np.random.random((3,100))

In [4]:
ipyvolume.quickscatter(x,y,z,size=1, marker='sphere')

In [6]:
from sys import path

In [7]:
path

['/Users/jnaiman/is445_spring2022/week14',
 '/Users/jnaiman/opt/anaconda3/envs/DataViz/lib/python37.zip',
 '/Users/jnaiman/opt/anaconda3/envs/DataViz/lib/python3.7',
 '/Users/jnaiman/opt/anaconda3/envs/DataViz/lib/python3.7/lib-dynload',
 '',
 '/Users/jnaiman/opt/anaconda3/envs/DataViz/lib/python3.7/site-packages',
 '/Users/jnaiman/opt/anaconda3/envs/DataViz/lib/python3.7/site-packages/IPython/extensions',
 '/Users/jnaiman/.ipython']

In [8]:
path.append('/Users/jnaiman/Downloads/')
path

['/Users/jnaiman/is445_spring2022/week14',
 '/Users/jnaiman/opt/anaconda3/envs/DataViz/lib/python37.zip',
 '/Users/jnaiman/opt/anaconda3/envs/DataViz/lib/python3.7',
 '/Users/jnaiman/opt/anaconda3/envs/DataViz/lib/python3.7/lib-dynload',
 '',
 '/Users/jnaiman/opt/anaconda3/envs/DataViz/lib/python3.7/site-packages',
 '/Users/jnaiman/opt/anaconda3/envs/DataViz/lib/python3.7/site-packages/IPython/extensions',
 '/Users/jnaiman/.ipython',
 '/Users/jnaiman/Downloads/']

In [9]:
from solverlibs import read_in_galaxy_data

In [10]:
galaxy_file ='/Users/jnaiman/Downloads/galaxyFiles/outarrsnap_001_fac1n3.txt'
t_h, r_h, v_h, e_h, N, part_type = read_in_galaxy_data(galaxy_file)
# t_h = time
# r_h = x,y,z positional vector for each timestep and each particle
# v_h = vx,vy,vz for each particle, each timestep
# e_h = energy (physics element)
# N = number of particles
# part_type = type of particle

In [11]:
r_h.shape

(82, 3, 500)

In [12]:
N

82

In [14]:
np.unique(part_type)

array([1., 2.])

In [15]:
tstep = 0

# r_h [particles, x/y/z, time step out of 500]
x,y,z = r_h[:,0,tstep], r_h[:,1,tstep], r_h[:,2,tstep]